Show how to use pretrained assertion status

In [1]:
from google.colab import drive
drive.mount("/content/gdrive/")
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/SparkNLP/utils')
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/
gdrive	sample_data


In [59]:
drivepath = "gdrive/My Drive/Colab Notebooks/SparkNLP/utils"
biotrainset = "con_rest_train.bio"
gloveemb = "glove.6B.300d.txt"
modelsavefilename="ner_dl_model_aug"
print("Driver path file name :"+biofile)
print("BIO file name :"+biotrainset)
print("Glovec file name :"+gloveemb)
print("modelsavefilename file name :"+modelsavefilename)
!cp "gdrive/My Drive/Colab Notebooks/SparkNLP/utils/con_rest_train.bio" "con_rest_train.bio"
!cp "gdrive/My Drive/Colab Notebooks/SparkNLP/utils/glove.6B.300d.txt" "glove.6B.300d.txt"
!cp "gdrive/My Drive/Colab Notebooks/SparkNLP/utils/blstm_17_300_128_37.pb" "blstm_17_300_128_37.pb"



Driver path file name :gdrive/My Drive/Colab Notebooks/SparkNLP/utilscon_rest_train.bio
BIO file name :con_rest_train.bio
Glovec file name :glove.6B.300d.txt
modelsavefilename file name :ner_dl_model_aug
cp: target 'biotrainset' is not a directory
cp: cannot stat 'drivepath/glove.6B.300d.txt': No such file or directory
cp: cannot stat 'drivepath/blstm_17_300_128_37.pb': No such file or directory


In [3]:
!ls

blstm_17_300_128_37.pb	gdrive		   sample_data
con_rest_train.bio	glove.6B.300d.txt


In [4]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.3

# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.2.2

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)
     |████████████████████████████████| 215.6MB 61kB/s 
     |████████████████████████████████| 204kB 43.4MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.3-py2.py3-none-any.whl size=215964963 sha256=c2bb95c3052e53e1f46a4f66717161b5dd5834ff8df05d3149f64d6cac12bce4
  Stored in directory: /root/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark


In [0]:
import sys

from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel

import sparknlp
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import ResourceDownloader

from pathlib import Path

if sys.version_info[0] < 3:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve    

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Global DEMO - Spark NLP Enterprise 2.3.4") \
    .master("local[*]") \
    .config("spark.rdd.compress","true") \
    .config("spark.driver.memory","8G") \
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "600M") \
    .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:2.3.4") \
    .getOrCreate()
#spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)


2.2.2
Spark NLP version:  None
Apache Spark version:  2.4.3


Create some data for testing purposes

In [0]:
from pyspark.sql import Row
R = Row('sentence', 'start', 'end')
test_data = spark.createDataFrame([R('Peter is a good person, and he was working at IBM',0,1)])

Create a custom pipeline

In [8]:
!ls
from sparknlp.training import CoNLL
training_data = CoNLL().readDataset(spark, 'con_rest_train.bio')
training_data.show()

blstm_17_300_128_37.pb	gdrive		   sample_data
con_rest_train.bio	glove.6B.300d.txt
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|2 star restaurant...|[[document, 0, 38...|[[document, 0, 36...|[[token, 0, 0, 2,...|[[pos, 0, 0, , [w...|[[named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [0]:
import time

documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddings = WordEmbeddings() \
        .setInputCols(["document", "token"])\
        .setOutputCol("embeddings")\
        .setEmbeddingsSource("glove.6B.300d.txt", 300, "text")

#spell = NorvigSweetingModel.pretrained() \
#    .setInputCols(["token"]) \
#    .setOutputCol("spell")

ner_dl = NerDLApproach()\
    .setInputCols(["document", "token","embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(2)\
    .setRandomSeed(0)\
    .setVerbose(2)\
    .setBatchSize(9)\
    .setGraphFolder("gdrive/My Drive/Colab Notebooks/SparkNLP/utils/Graphs")\
    .setEnableOutputLogs(True)

converter = NerConverter()\
  .setInputCols(["document", "token", "ner"])\
  .setOutputCol("ner_converter")

finisher = Finisher() \
    .setInputCols(["ner", "ner_converter"]) \
    .setIncludeMetadata(True)

pipeline_fast_dl = Pipeline(stages = [
    documentAssembler, 
    tokenizer, 
    embeddings, 
    ner_dl,
    converter,
    finisher])

Now let's use these pipelines and see the results

In [22]:
%%time

start = time.time()
print("Start fitting")
prediction_model = pipeline_fast_dl.fit(training_data)
print("Fitting is ended")
print (time.time() - start)

Start fitting
Fitting is ended
912.3336133956909
CPU times: user 218 ms, sys: 56.4 ms, total: 275 ms
Wall time: 15min 12s


In [23]:
prediction_data = spark.createDataFrame([["Maria is a nice place."],["any bbq places open before 5 nearby"]]).toDF("text")
prediction_data.show()
prediction_model.transform(prediction_data).show(truncate=False)

+--------------------+
|                text|
+--------------------+
|Maria is a nice p...|
|any bbq places op...|
+--------------------+

+-----------------------------------+---------------------+----------------------+---------------------------------------------------------------------------------------------------+-------------------------------+
|text                               |finished_ner         |finished_ner_converter|finished_ner_metadata                                                                              |finished_ner_converter_metadata|
+-----------------------------------+---------------------+----------------------+---------------------------------------------------------------------------------------------------+-------------------------------+
|Maria is a nice place.             |[O, O, O, O, O, O]   |[]                    |[[word, Maria], [word, is], [word, a], [word, nice], [word, place], [word, .]]                     |[]                             |
|

In [0]:
prediction_model.write().overwrite().save("ner_dl_model_aug")

In [0]:
!cp -r 'ner_dl_model_aug' 'gdrive/My Drive/Colab Notebooks/SparkNLP/utils/ner_dl_model_aug'

In [0]:
from pyspark.ml import PipelineModel, Pipeline

loaded_prediction_model = PipelineModel.read().load(ner_dl_model_aug)

In [49]:
prediction_data = spark.createDataFrame([["Maria is a nice place."],["any bbq places open before 5 nearby"],["2 star restaurants with inside dining"]]).toDF("text")
prediction_data.show()
loaded_prediction_model.transform(prediction_data).show(truncate=False)
prediction = loaded_prediction_model.transform(prediction_data)
prediction.select("finished_ner_metadata").show(truncate=False)
prediction.select("finished_ner").show(truncate=False)
prediction.select("finished_ner_converter_metadata").show(truncate=False)
prediction.select("finished_ner_converter").show(truncate=False)
#prediction.select("ner").show(truncate=False)

+--------------------+
|                text|
+--------------------+
|Maria is a nice p...|
|any bbq places op...|
|2 star restaurant...|
+--------------------+

+-------------------------------------+---------------------+----------------------+---------------------------------------------------------------------------------------------------+-------------------------------+
|text                                 |finished_ner         |finished_ner_converter|finished_ner_metadata                                                                              |finished_ner_converter_metadata|
+-------------------------------------+---------------------+----------------------+---------------------------------------------------------------------------------------------------+-------------------------------+
|Maria is a nice place.               |[O, O, O, O, O, O]   |[]                    |[[word, Maria], [word, is], [word, a], [word, nice], [word, place], [word, .]]                     |[] 

In [39]:
from sparknlp.base import LightPipeline

lp = LightPipeline(loaded_prediction_model)
result = lp.annotate("Peter is a good person.")
for e in list(zip(result['token'], result['ner']))[:10]:
    print(e)

Py4JError: ignored

In [40]:
for stage in loaded_prediction_model.stages:
    print(stage)
print(loaded_prediction_model.stages[-1].stages)

DocumentAssembler_44ad655ebd78
REGEX_TOKENIZER_62aa1d44b242
WORD_EMBEDDINGS_MODEL_31fbcf792801
NerDLModel_41805337f0d2
NerConverter_04d283d06e00
Finisher_11c085484ffc


AttributeError: ignored